## Import

In [ ]:
!pip install transformers
from typing import Dict, List
import csv
import torch
from transformers import (
    EncoderDecoderModel,
    GPT2Tokenizer as BaseGPT2Tokenizer,
    PreTrainedTokenizer, BertTokenizerFast,
    PreTrainedTokenizerFast,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Trainer
)
from torch.utils.data import DataLoader
from transformers.models.encoder_decoder.modeling_encoder_decoder import EncoderDecoderModel

encoder_model_name = "cl-tohoku/bert-base-japanese-v2"
decoder_model_name = "nlpotato/kogpt2_chatbot_social_media-e10"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 100.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 113.7 MB/s eta 0:00:00


In [ ]:
!pip install wandb==0.9.7
import wandb
wandb.login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device, torch.cuda.device_count()

(device(type='cuda', index=0), 1)

In [ ]:
class GPT2TokenizerFast(PreTrainedTokenizerFast):
    def build_inputs_with_special_tokens(self, token_ids: List[int]) -> List[int]:
        return token_ids + [self.eos_token_id]        

'''
encoder_model_name = "cl-tohoku/bert-base-japanese-v2"
decoder_model_name = "nlpotato/kogpt2_chatbot_social_media-e10"
'''
src_tokenizer = BertTokenizerFast.from_pretrained(encoder_model_name)
trg_tokenizer = GPT2TokenizerFast.from_pretrained(decoder_model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'GPT2TokenizerFast'.


In [ ]:
li = []
a = list(src_tokenizer.get_vocab())

In [ ]:
# [unused]
# <unused>
if '[unused]' or '<unused>' in a:
  print("true")
else:
  print("False")

print(a(<unused>))

SyntaxError: ignored

In [ ]:
trg_tokenizer.build_inputs_with_special_tokens(trg_tokenizer("안녕", return_attention_mask=False)['input_ids'])

[25906, 1]

## Data

In [ ]:
class PairedDataset:
    def __init__(self, 
        src_tokenizer: PreTrainedTokenizerFast, tgt_tokenizer: PreTrainedTokenizerFast,
        file_path: str
    ):
        self.src_tokenizer = src_tokenizer
        self.trg_tokenizer = tgt_tokenizer
        with open(file_path, 'r') as fd:
            reader = csv.reader(fd)
            next(reader)
            self.data = [row for row in reader]

    def __getitem__(self, index: int) -> Dict[str, torch.Tensor]:
        src, trg = self.data[index]
        embeddings = self.src_tokenizer(src, return_attention_mask=False, return_token_type_ids=False)
        embeddings['labels'] = self.trg_tokenizer.build_inputs_with_special_tokens(self.trg_tokenizer(trg, return_attention_mask=False)['input_ids'])

        return embeddings

    def __len__(self):
        return len(self.data)
train_dataset = PairedDataset(src_tokenizer, trg_tokenizer, '/content/drive/MyDrive/구름/Project_3/life_train.csv')
eval_dataset = PairedDataset(src_tokenizer, trg_tokenizer, '/content/drive/MyDrive/구름/Project_3/life_val.csv')        

In [ ]:
len(train_dataset)
# 일상생활, 채팅

1144904

## Model

In [ ]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    encoder_model_name,
    decoder_model_name,
    pad_token_id=trg_tokenizer.bos_token_id,
)
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at nlpotato/kogpt2_chatbot_soc

In [ ]:
# # for custom training

# collate_fn = DataCollatorForSeq2Seq(src_tokenizer, model)

# train_dataloader = DataLoader(
#       train_dataset,
#       batch_size = 32,
#       collate_fn = collate_fn,
#       num_workers = 2,
#       shuffle= True,
# )
# val_dataloader = DataLoader(
#       train_dataset,
#       batch_size = 16,
#       collate_fn = collate_fn,
#       num_workers = 2,
#       shuffle= False,
# )

In [ ]:
# for Trainer

collate_fn = DataCollatorForSeq2Seq(src_tokenizer, model)
wandb.init(project="temp", name='enc_dec_110m')

arguments = Seq2SeqTrainingArguments(
    output_dir='dump2',
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    warmup_ratio=0.1,
    gradient_accumulation_steps=4,
    save_total_limit=5,
    dataloader_num_workers=1,
    fp16=True,
    load_best_model_at_end=True,
    report_to='wandb'
)

trainer = Trainer(
    model,
    arguments,
    data_collator=collate_fn,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

wandb: Wandb version 0.13.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
Using cuda_amp half precision backend


## Training

In [ ]:
# from transformers import AutoModel
# model = EncoderDecoderModel.from_pretrained("/content/drive/MyDrive/구름/Project_3/dump/best_model")

In [ ]:
trainer.train()

model.save_pretrained("dump2/best_model")

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1144904
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 26832
  Number of trainable parameters = 264738816
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Wandb version 0.13.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `

Epoch,Training Loss,Validation Loss
0,1.042000,0.925313
1,0.793500,0.773699
2,0.644600,0.727534


***** Running Evaluation *****
  Num examples = 143113
  Batch size = 16
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Saving model checkpoint to dump2/checkpoint-8944
Configuration saved in dump2/checkpoint-8944/config.json
Configuration saved in dump2/checkpoint-8944/generation_config.json
Model weights saved in dump2/checkpoint-8944/pytorch_model.bin
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.8/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-de

## 번역

In [ ]:
from transformers import AutoModel
model = EncoderDecoderModel.from_pretrained("/content/drive/MyDrive/구름/Project_3/dump2/best_model")

In [ ]:
# text = "お新香盛合せ","さけ"

file_path = "/content/drive/MyDrive/구름/Project_3/일본어 음식.txt"
with open(file_path, 'r') as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        jp = line.strip().split(",")[0].strip()

        embeddings = src_tokenizer(jp, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
        embeddings = {k: v for k, v in embeddings.items()}
        output = model.generate(**embeddings)[0, 1:-1]
        print(trg_tokenizer.decode(output.cpu()))

In [ ]:
text = "宅配便の問題だと確認されましたか？"
embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
embeddings = {k: v for k, v in embeddings.items()}
output = model.generate(**embeddings)[0, 1:-1]
print(trg_tokenizer.decode(output.cpu()))

택배문제인걸로 확인되셨습니까?


##f1 score

In [ ]:
a =  pd.read_excel("/content/drive/MyDrive/구름/Project_3/새로만든test파일.xlsx")
a.to_csv("/content/drive/MyDrive/구름/Project_3/new_test.csv", index=False)

In [ ]:
from tqdm import tqdm
from statistics import mean
from nltk.translate.bleu_score import sentence_bleu
import pandas as pd
bleu = []
f1 = []


with torch.no_grad(), open('/content/drive/MyDrive/구름/Project_3/new_test.csv', 'r') as fd:
    reader = csv.reader(fd)
    next(reader)
    datas = [row for row in reader]

    for data in tqdm(datas, "Testing"):
        input, label = data
        embeddings = src_tokenizer(input, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
        embeddings = {k: v for k, v in embeddings.items()}
        with torch.no_grad():
            output = model.generate(**embeddings)[0, 1:-1]
        preds = trg_tokenizer.decode(output.cpu())

        bleu.append(sentence_bleu([label.split()], preds.split(), weights=[1,0,0,0]))

Testing:   0%|          | 0/400 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis conta

In [ ]:
'''
life_test.csv 앞에서 1000개 테스트 진행
encoder_model_name = "cl-tohoku/bert-base-japanese-v2"
decoder_model_name = "nlpotato/kogpt2_chatbot_social_media-e10" 모델
'''
print(f"Bleu score: {mean(bleu)}")

Bleu score: 0.5273088249082412


## custom 학습

In [ ]:
# # start, end 지점 필요!

# import os
# from statistics import mean
# from tqdm import tqdm, tqdm_notebook
# import torch.nn.functional as F
# from torch.nn.utils import clip_grad_norm_

# wandb.init(project='KRbert-QA', name='KRBert_QA/epoch3/eta_max=1e-4/Adam/Cosinerestart')

# # os.makedirs('dump', exist_ok=True)
# train_step_losses = []
# dev_step_losses = []
# train_losses = []
# dev_losses = []
# lowest_dev_loss = 9999

# epochs = 3
# step = 0

# for epoch in range(epochs):
#     print("Epoch", epoch)
#     # Training
#     running_loss = 0.
#     losses = []
#     progress_bar = tqdm(train_dataloader, desc='Train')
#     for input in progress_bar:
#         model.train()
#         input = input.to(device)
        
#         outputs = model(**input)
#         loss = outputs.loss
#         loss.backward()
#         running_loss += loss.item()
#         del input, loss

#         clip_grad_norm_(model.parameters(), max_norm=1.)
#         optimizer.step()
#         scheduler.step()
#         optimizer.zero_grad(set_to_none=True)

#         train_step_losses.append(running_loss)
#         losses.append(running_loss)
#         running_loss = 0.
#         progress_bar.set_description(f"Train - Loss: {losses[-1]:.3f}")
#         wandb.log({"Train Loss": losses[-1]})
#     train_losses.append(mean(losses))
#     print(f"train score: {train_losses[-1]:.3f}")

#         # if step % 1024:
#         # Evaluation
#     val_losses = []
#     for input in tqdm(val_dataloader, desc="Evaluation"):
#         model.eval()        
#         input = input.to(device)
        
#         with torch.no_grad():
#             outputs = model(**input)
#         loss = outputs.loss

#         dev_step_losses.append(loss.item())
#         val_losses.append(loss.item())
#         wandb.log({"Valid_loss": mean(dev_step_losses)}) # valid loss
#         del input, loss
#     dev_losses.append(mean(dev_step_losses))
#     print(f"Evaluation score: {dev_losses[-1]:.3f}")

#     if lowest_dev_loss > dev_losses[-1]:                    
#         lowest_dev_loss = dev_losses[-1]
#         # torch.save(model.state_dict(), "./")
#         model.save_pretrained(f'./')
#         # model.train()

#     # wandb.log({"Acc": acc}) # acc